In [18]:
#!curl -O http://www.atarimania.com/roms/Roms.rar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 10.6M    0  8491    0     0   8489      0  0:21:50  0:00:01  0:21:49  8491
  1 10.6M    1  130k    0     0  64277      0  0:02:53  0:00:02  0:02:51 64326
  3 10.6M    3  389k    0     0   131k      0  0:01:22  0:00:02  0:01:20  131k
  5 10.6M    5  641k    0     0   165k      0  0:01:05  0:00:03  0:01:02  165k
  8 10.6M    8  961k    0     0   193k      0  0:00:56  0:00:04  0:00:52  193k
 11 10.6M   11 1288k    0     0   212k      0  0:00:51  0:00:06  0:00:45  253k
 14 10.6M   14 1541k    0     0   222k      0  0:00:48  0:00:06  0:00:42  291k
 17 10.6M   17 1860k    0     0   232k      0  0:00:46  0:00:07  0:00:39  292k
 19 10.6M   19 2114k    0     0   237k      0  0:00:45  0:00:08  0:00:37  293k
 22 10.6M   22 2434k    0     0   243k      0  0:00

In [17]:
#!7z x Roms.rar


7-Zip 19.00 (x64) : Copyright (c) 1999-2018 Igor Pavlov : 2019-02-21

Scanning the drive for archives:
1 file, 11128004 bytes (11 MiB)

Extracting archive: Roms.rar
--
Path = Roms.rar
Type = Rar5
Physical Size = 11128004
Solid = -
Blocks = 2
Encrypted = -
Multivolume = -
Volumes = 1


No files to process
Everything is Ok

Files: 0
Size:       0
Compressed: 11128004


In [3]:
from stable_baselines.common.atari_wrappers import make_atari, wrap_deepmind
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
from ale_py import ALEInterface
ale = ALEInterface()

In [19]:
import ale_py.roms as roms

In [20]:
print(roms.__all__)

['Tetris']


In [26]:
from ale_py.roms import Tetris

In [27]:
ale.loadROM(Tetris)

In [7]:
import gym


In [36]:
from stable_baselines.common.cmd_util import make_atari_env


ModuleNotFoundError: No module named 'tensorflow.contrib'

In [38]:
make_atari_env('Tetris', num_env=16, seed=0)

AttributeError: module 'tensorflow' has no attribute 'set_random_seed'

In [30]:
env = make_atari('Tetris')

Error: Attempted to look up malformed environment ID: b'Tetris'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)

In [31]:
env = gym.make('ALE/Tetris')

Error: Attempted to look up malformed environment ID: b'ale/Tetris'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)

In [22]:
env = gym.make('tetris')

Error: Attempted to look up malformed environment ID: b'tetris'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)

In [16]:
#ale-import-roms roms/

SyntaxError: invalid syntax (Temp/ipykernel_8148/2249535004.py, line 1)

In [13]:
env = make_atari('Breakout')

Error: Attempted to look up malformed environment ID: b'Breakout'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)

In [14]:
# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 10000

# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari("BreakoutNoFrameskip-v4")
# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)
env.seed(seed)

Error: We're Unable to find the game "Breakout". Note: Gym no longer distributes ROMs. If you own a license to use the necessary ROMs for research purposes you can download them via `pip install gym[accept-rom-license]`. Otherwise, you should try importing "Breakout" via the command `ale-import-roms`. If you believe this is a mistake perhaps your copy of "Breakout" is unsupported. To check if this is the case try providing the environment variable `PYTHONWARNINGS=default::ImportWarning:ale_py.roms`. For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management

In [ ]:
num_actions = 4


def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(84, 84, 4,))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)


# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(84, 84, 4,))